<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2024/blob/main/DynamicDistillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from scipy.integrate import solve_ivp
import numpy as np
from plotly.subplots import make_subplots

In [20]:
#0  Condenser
#1
#2  Nr-1
#3  Feed Stage
#4
#5
#6  Nr+Ns-1
#7  Nr+Ns:  Reboiler

Nr = 3
Ns = 4

F=1.5  #mol/s
z=0.4
q = 0.7
R=2.
D=F/2
alpha=2.
stage_holdup=0.5
feedstage_holdup = 0.75
condenser_holdup = 5.
reboiler_holdup = 5.
m_holdup=np.r_[condenser_holdup, np.full(Nr-1, stage_holdup), feedstage_holdup, np.full(Ns-1, stage_holdup) ,
               reboiler_holdup]
W=1.  # 1/(s mol**0.5)

#initial conditions
m0 = m_holdup*1.05  #initial number moles on each stage is set slightly higher than holdup to yield a little flow
m0[0] = (((R+1)*D)/W)**(2/3)+condenser_holdup
x0 = np.full_like(m0, z)  #fill each stage with feed composition


In [33]:
def ramp(t, a, b):
    if t<a:
        return 0.
    if t>b:
        return 1.
    return (t-a)/(b-a)


In [43]:
fig=make_subplots()
tplot = np.linspace(0,100)
yplot = [2*ramp(t,25,75)+1 for t in tplot]

fig.add_scatter(x=tplot, y=yplot)

In [44]:
def rhs(t, v):
    R=2.
    D=ramp(t, 0,50)*F/2
    m,x = np.split(v,2)
    y=alpha*x/(1-x+alpha*x)
    dm = np.zeros_like(m)
    dmx = np.zeros_like(m)
    V = np.zeros_like(m)

    L =  np.where(m<m_holdup, 0., W*(m-m_holdup)**1.5)

    L[0]=R*D

    Vrec = L[0] + D

    V[1:Nr+1] = Vrec  #constant vapor flow rate in rectifying section
    V[Nr+1:] = Vrec - F*(1-q)  #constant vapor flow rate in stripping section

    dm[1:-1] =  L[:-2] - L[1:-1] + V[2:] - V[1:-1]
    #  1to6      0to5      1to6     2to7     1to6
    dm[Nr]+=F
    dm[0] = V[1] - L[0] - D
    dm[-1] = L[-2] - V[-1] - L[-1]

    dmx[1:-1] =  x[:-2]*L[:-2] - x[1:-1]*L[1:-1] + y[2:]*V[2:] - y[1:-1]*V[1:-1]
    dmx[Nr]+=z*F
    dmx[0] = y[1]*V[1] - x[0]*L[0] - x[0]*D
    dmx[-1] = x[-2]*L[-2] - y[-1]*V[-1] - x[-1]*L[-1]

    dx = (dmx - x*dm)/m
    return np.r_[dm, dx]

In [45]:
tend=1000
res=solve_ivp(rhs, (0, tend), np.r_[m0, x0], method='Radau', dense_output=True)
res

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  7.376e-02 ...  6.916e+02  1.000e+03]
        y: [[ 6.717e+00  6.717e+00 ...  6.717e+00  6.717e+00]
            [ 5.250e-01  5.248e-01 ...  1.810e+00  1.810e+00]
            ...
            [ 4.000e-01  4.000e-01 ...  1.590e-01  1.590e-01]
            [ 4.000e-01  4.011e-01 ...  1.009e-01  1.009e-01]]
      sol: <scipy.integrate._ivp.common.OdeSolution object at 0x7d3821bbcf10>
 t_events: None
 y_events: None
     nfev: 241
     njev: 9
      nlu: 52

In [47]:
tplot=np.linspace(0,tend,100)
fig = make_subplots(rows=1, cols=2)
ms,xs = np.split(res.sol(tplot),2,axis=0)
for i,m in enumerate(ms):
    fig.add_scatter(x=tplot, y=m, row=1,col=1, name=f'moles {i}')

for i,x in enumerate(xs):
    fig.add_scatter(x=tplot, y=x, row=1,col=2, name=f'molfrac {i}')

fig.update_layout(width=800, height=400, template='plotly_dark')



In [51]:
xss=xs[:,-1]
xss

array([0.69911851, 0.53741911, 0.41976775, 0.34488553, 0.28745992,
       0.22339144, 0.15904163, 0.10088178])

In [55]:
xplot=np.linspace(0,1,100)


fig=make_subplots()
fig.add_scatter(x=[0,1],y=[0,1],mode='lines',line_color='grey')
fig.add_scatter(x=xplot, y=alpha*xplot/(1-xplot+alpha*xplot), line_color='green')
fig.update_layout(width=500,height=500, template='plotly_dark')

In [ ]:
fig= make_subplots(rows=3, cols=4)
for mx in res.sol(tplot).T:
    m,x = np.split(mx,2)


In [ ]:
res.sol(tplot).T.shape

(100, 16)